In [ ]:
# ---------------------------------------------------------------
#  N‑PX filing loader – SageMaker / S3 / RDS edition (April 2025)
# ---------------------------------------------------------------
"""
Runs inside an **Amazon SageMaker** notebook (or Studio) that sits in a
VPC.  Filings live in an S3 bucket _in another VPC_, and results are
written to a PostgreSQL RDS instance (also usually in that VPC or peered
one).

👷 **What this file does**
1.  Streams each `*.txt` filing directly from S3 (zero local disk I/O)
   via `s3fs`.
2.  Parses, batches, and inserts into RDS exactly as before – we reuse
   all helper logic.
3.  Detects at runtime whether we are reading from **local disk**
   (`FILINGS_DIR`) or an S3 bucket (env var `S3_BUCKET`).  Same business
   code works in either context.

👮 **Infra prerequisites** (one‑time):
- The SageMaker execution‑role must have `s3:GetObject` permission on the
  bucket/prefix that holds the filings.
- The notebook must run in a subnet that can reach the RDS endpoint
  (same VPC, VPC‑peered, or via AWS PrivateLink).  Security group allows
  outbound 5432.
- RDS has a SG rule allowing inbound 5432 from the notebook’s security
  group.
- Environment variables `PGHOST PGPORT PGUSER PGPASSWORD PGDATABASE`
  should be either stored as **Secrets Manager** secrets retrieved by the
  notebook or pre‑set via the lifecycle config.  For demo they are still
  prompted if missing.

Install dependencies in a SageMaker conda cell:
```bash
pip install s3fs psycopg2-binary tqdm python-dotenv lxml
```
"""

# ---------------------------------------------------------------
# 1 · Imports & configuration
# ---------------------------------------------------------------
import os
import re
import logging
import datetime as dt
import getpass
import tempfile
from pathlib import Path
from collections import defaultdict
import functools

import boto3
import s3fs                     # <‑‑ new
import psycopg2
from psycopg2.extras import execute_values, DictCursor
from lxml import etree as ET
from tqdm.auto import tqdm      # auto picks notebook/console
from decimal import Decimal
from dotenv import load_dotenv

# --- logging --------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%H:%M:%S",
)

# --- environment ----------------------------------------------------------
FILINGS_DIR = Path("npx_filings")          # ignored when S3 is used
S3_BUCKET  = os.getenv("S3_BUCKET")        # if set ➟ use S3
S3_PREFIX  = os.getenv("S3_PREFIX", "")   # optional key prefix

# --- SEC regex helpers ----------------------------------------------------
DATE_FMTS    = ("%Y-%m-%d", "%m/%d/%Y", "%m-%d-%Y", "%Y%m%d")
DEC_CLEAN_RE = re.compile(r"[^\d\.\-]")
SEC_HEADER_RE = re.compile(
    r"ACCESSION\s+NUMBER:\s*(?P<acc>[^\r\n]+).*?" \
    r"FILED\s+AS\s+OF\s+DATE:\s*(?P<filed>\d{8})",
    re.I | re.S,
)
XML_BLOCK_RE = re.compile(r"(<\?xml.*?</edgarSubmission>)", re.I | re.S)
XML_PARSER   = ET.XMLParser(recover=True, encoding="utf-8", huge_tree=True)

# --- credentials ----------------------------------------------------------
load_dotenv()
for var in ("PGHOST", "PGPORT", "PGUSER", "PGPASSWORD", "PGDATABASE"):
    if not os.getenv(var):
        os.environ[var] = (
            getpass.getpass(f"{var}: ") if var == "PGPASSWORD" else input(f"{var}: ")
        )

# ---------------------------------------------------------------
# 2 · Helper functions (unchanged except Decimal + S3)
# ---------------------------------------------------------------

@functools.lru_cache(maxsize=1024)
def parse_date(s: str | None):
    if not s:
        return None
    s = s.strip()
    for fmt in DATE_FMTS:
        try:
            return dt.datetime.strptime(s, fmt).date()
        except ValueError:
            continue
    logging.warning("Un‑parsable date: %s", s)
    return None


def clean_decimal(s: str | None):
    if not s:
        return None
    txt = DEC_CLEAN_RE.sub("", s)
    try:
        return Decimal(txt) if txt else None
    except ArithmeticError:
        return None


def get(node, xp, sl=None):
    res = node.xpath(xp)
    if not res:
        return ""
    txt = res[0] if isinstance(res[0], str) else res[0].text or ""
    txt = txt.strip()
    return txt[:sl] if sl else txt

# ---------------------------------------------------------------
# 3 · SEC header & XML extraction (text‑based)
# ---------------------------------------------------------------

def extract_sec_header_from_text(txt: str):
    head = txt[:4000]
    m = SEC_HEADER_RE.search(head)
    if not m:
        return {"accession_number": "", "date_filed": None}
    return {
        "accession_number": m.group("acc").strip(),
        "date_filed": dt.datetime.strptime(m.group("filed"), "%Y%m%d").date(),
    }


def xml_blocks_from_text(txt: str):
    return XML_BLOCK_RE.findall(txt)

# ---------------------------------------------------------------
# 4 · All parsing & DB helpers (identical to previous patch)
# ---------------------------------------------------------------
#   – for brevity not repeated here; see earlier code –
#   keep: parse_edgar_submission, parse_institutional_managers,
#         parse_series_info, parse_proxy_tables, pg_conn, upsert_category

# (Insert the same definitions from the prior patched file.)

# ---------------------------------------------------------------
# 5 · Loader that consumes **raw text** (works for S3 or disk)
# ---------------------------------------------------------------

VOTE_COLS = [
    "form_id", "issuer_name", "cusip", "isin", "figi",
    "meeting_date", "vote_description", "proposed_by",
    "shares_voted", "shares_on_loan",
    "vote_cast", "vote_cast_shares", "management_rec",
    "other_notes",
]

def load_filing_text(filename: str, txt: str, conn, cat_cache):
    hdr   = extract_sec_header_from_text(txt)
    frags = xml_blocks_from_text(txt)
    if not frags:
        logging.warning("%s – no <edgarSubmission> blocks", filename)
        return

    with conn.cursor() as cur:
        form_map, mgr_map, ser_map = {}, defaultdict(dict), defaultdict(dict)
        for frag in frags:
            root = ET.fromstring(frag.encode(), parser=XML_PARSER)
            #  --- IDENTICAL inner logic as before (forms/managers/series/votes) ---
            #  Copy the body of load_filing() from the patched version, but
            #  replace 'path.name' with 'filename' in log messages.
            # ------------------------------------------------------------------
            #  (omitted here to keep this snippet concise.)
            # ------------------------------------------------------------------
        logging.info("%s – committed", filename)

# ---------------------------------------------------------------
# 6 · Iterators over local disk or S3
# ---------------------------------------------------------------

def iter_local_files() -> tuple[str, str]:
    for path in sorted(FILINGS_DIR.glob("*.txt")):
        yield path.name, path.read_text("utf-8", "replace")


def iter_s3_files(bucket: str, prefix: str) -> tuple[str, str]:
    fs = s3fs.S3FileSystem(anon=False)     # IAM role supplies creds
    keys = fs.ls(f"{bucket}/{prefix}")
    for key in tqdm(keys, desc="S3 keys"):
        # key is like 'bucket/prefix/file.txt'; we need only the object key
        obj_key = "/".join(key.split("/")[1:]) if key.startswith(bucket) else key
        with fs.open(f"{bucket}/{obj_key}", "r") as f:
            yield obj_key.split("/")[-1], f.read()

# ---------------------------------------------------------------
# 7 · Orchestrator – chooses S3 vs local and streams to RDS
# ---------------------------------------------------------------

def run_loader():
    if S3_BUCKET:
        logging.info("Running in S3 mode: bucket=%s prefix='%s'", S3_BUCKET, S3_PREFIX)
        file_iter = iter_s3_files(S3_BUCKET, S3_PREFIX)
    else:
        file_iter = iter_local_files()

    with pg_conn() as conn:
        cat_cache = {}
        for fname, txt in tqdm(list(file_iter), desc="Filings"):
            try:
                load_filing_text(fname, txt, conn, cat_cache)
                conn.commit()
            except Exception:
                logging.exception("‼️  %s failed – rolled back", fname)
                conn.rollback()


if __name__ == "__main__":
    run_loader()


You only need to “splice in” whatever the **other notebook does that our loader doesn’t already do**.\
From what you’ve said, that extra value is *discovering which S3 objects are actually N‑PX filings* among a lot of miscellaneous stuff.\
Everything else—parsing, batching, writing to RDS—is already in the patched loader.

Below is a simple roadmap plus a 10‑line code stub that shows exactly where to hook the discovery logic in.\
(You can keep both notebooks separate: just `import` the discovery function, or paste it into the small area marked ▸.)

---

## 1 · Identify the one function you want

Open the other notebook and find (or create) a helper that returns the keys you need, e.g.:

```python
def list_npx_keys(bucket: str, prefix: str) -> list[str]:
    \"\"\"Return *only* the S3 keys that are text N‑PX filings.\"\"\"
    ...
```

Typical filters inside might be:

- key ends with `.txt`
- key path includes `/npx/`
- object metadata tag `sec-form = N-PX`

---

## 2 · Import (or copy) that function into the loader

### Option A – import the notebook as a module

- In the other notebook add:

  ```python
  # at bottom
  if __name__ == "__main__":
      pass
  ```

- Save it as `npx_s3_discovery.py` in the same directory.

- In our loader add:

  ```python
  from npx_s3_discovery import list_npx_keys
  ```

### Option B – copy the few lines directly

Paste the function into **Section 6** of the loader (right before the iterators).

---

## 3 · Replace the simple iterator

Open the canvas loader and swap the body of `iter_s3_files()` for this:

```python
def iter_s3_files(bucket: str, prefix: str) -> tuple[str, str]:
    \"\"\"Yield (filename, file‑text) only for true N‑PX filings.\"\"\"
    fs = s3fs.S3FileSystem(anon=False)
    keys = list_npx_keys(bucket, prefix)          # ▸ your smarter filter
    for key in tqdm(keys, desc=\"N‑PX keys\"):
        with fs.open(f\"{bucket}/{key}\", \"r\") as f:
            yield key.split(\"/\")[-1], f.read()
```

That is literally the only edit you need—everything else stays the same.

---

## 4 · Run it

```python
%env S3_BUCKET=my-sec-bucket
%env S3_PREFIX=raw/sec-filings/
run_loader()           # progress bar should now show only true N‑PX files
```

---

### Quick checklist

| Step                  | What to confirm                                                                                |
| --------------------- | ---------------------------------------------------------------------------------------------- |
| Import works          | `from npx_s3_discovery import list_npx_keys` runs with no `ModuleNotFoundError`.               |
| Function returns keys | `print(list_npx_keys(bucket, prefix)[:3])` shows paths ending in `.txt`.                       |
| Loader picks them up  | The progress bar label changed from **“S3 keys”** to **“N‑PX keys”** and counts fewer objects. |

After that, the loader’s existing parsing/batching logic finishes the job and writes to RDS as before—no other integration required.
